In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.ensemble import  RandomForestClassifier
import pickle
import os

In [2]:
flight_data = (
    pd.read_csv('final_dataset_cleaned_v1.csv')
)

In [3]:
flight_data.shape

(12658696, 29)

In [4]:
flight_data.head(n = 3)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,TAIL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,...,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,coco,target
0,2020,1,1,1,3,2020-01-01,N951WN,ONT,"Ontario, CA",SFO,...,2020-01-01 18:00:00,12.2,-7.3,25.0,0.0,330.0,18.4,1017.3,3.0,1
1,2020,1,1,1,3,2020-01-01,N467WN,ONT,"Ontario, CA",SFO,...,2020-01-01 11:00:00,3.9,-3.4,59.0,0.0,210.0,5.4,1017.6,2.0,0
2,2020,1,1,1,3,2020-01-01,N7885A,ONT,"Ontario, CA",SJC,...,2020-01-01 20:00:00,12.8,-7.3,24.0,0.0,290.0,11.2,1015.8,2.0,0


In [5]:
flight_data['aircraft_type_new'] = [i.split('-')[0] for i in flight_data['Aircraft type']]

In [6]:
# list of numerical and categorical features to select 
l_numeric_cols = [ 'DEP_HOUR', 'temp', 'dwpt', 'rhum', 'prcp', 'wspd', 'pres', 'coco']
l_categorical_cols = ['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'aircraft_type_new']
target = "target"

In [7]:
# converting the categorical columns to categorical
for col in l_categorical_cols:
    flight_data[col] = flight_data[col].astype('category')

In [8]:
flight_data = flight_data[l_numeric_cols + l_categorical_cols + [target] + ['YEAR','FL_DATE']]

In [9]:
# getting dummu values for categorical columns
QUARTER_dummies = pd.get_dummies(flight_data['QUARTER'], prefix='QUARTER', drop_first=True)
MONTH_dummies = pd.get_dummies(flight_data['MONTH'], prefix='MONTH', drop_first=True)
DAY_OF_MONTH_dummies = pd.get_dummies(flight_data['DAY_OF_MONTH'], prefix='DAY_OF_MONTH', drop_first=True)
DAY_OF_WEEK_dummies = pd.get_dummies(flight_data['DAY_OF_WEEK'], prefix='DAY_OF_WEEK', drop_first=True)
Aircraft_type_dummies = pd.get_dummies(flight_data['aircraft_type_new'], prefix='Aircraft_type', drop_first=True)

In [10]:
# Aircraft_type_dummies.columns = ['Aircraft_{}'.format(i) for i in range(len(Aircraft_type_dummies.columns))]

In [11]:
flight_data = flight_data.drop(l_categorical_cols, axis=1)

In [12]:
dummy_flight_data = pd.concat([flight_data, QUARTER_dummies, MONTH_dummies,
                               DAY_OF_MONTH_dummies, DAY_OF_WEEK_dummies, Aircraft_type_dummies ], axis=1)

In [13]:
del Aircraft_type_dummies,QUARTER_dummies,MONTH_dummies,DAY_OF_MONTH_dummies,DAY_OF_WEEK_dummies

In [14]:
train = dummy_flight_data[dummy_flight_data['FL_DATE'] < "2020-08-01"]
test = dummy_flight_data[dummy_flight_data['FL_DATE'] >= "2020-08-01"]

In [15]:
del flight_data, dummy_flight_data

In [16]:
from imblearn.under_sampling import RandomUnderSampler

undersample = RandomUnderSampler(sampling_strategy='majority', random_state=42)
X_res, y_res = undersample.fit_resample(train.drop(["YEAR","target","FL_DATE"], axis=1).values,train[target])

In [17]:
del train

In [18]:
X_res1, y_res1 = undersample.fit_resample(X_res, y_res)
del X_res, y_res

In [19]:
X_res2, y_res2 = undersample.fit_resample(X_res1, y_res1)
del X_res1, y_res1

In [20]:
y_res2.value_counts()

0    253956
1    253956
2    253956
3    253956
Name: target, dtype: int64

In [22]:
train = pd.DataFrame(X_res2, columns=test.columns.drop(["YEAR","target","FL_DATE"]))

In [23]:
train['target'] = y_res2

In [24]:
train.to_csv('undersampled_training_data_combined.csv', index=False)

In [25]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(
    train.drop(["target"], axis=1).values,
     train[target]
     )

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [26]:
path = "./models"
filename = "combined_RF_undersample.mdl"

In [27]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(classifier, f, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
# predicting the test results 
y_pred = classifier.predict(test.drop(["YEAR","target","FL_DATE"], axis=1).values)

In [29]:
print(confusion_matrix(test[target],y_pred))

[[893668 397677 214829 253661]
 [ 34003  26435  13381  15537]
 [  8655   6064   4567   5902]
 [  4935   3448   3067   4659]]


In [30]:
print(classification_report(test[target],y_pred))

              precision    recall  f1-score   support

           0       0.95      0.51      0.66   1759835
           1       0.06      0.30      0.10     89356
           2       0.02      0.18      0.03     25188
           3       0.02      0.29      0.03     16109

    accuracy                           0.49   1890488
   macro avg       0.26      0.32      0.21   1890488
weighted avg       0.89      0.49      0.62   1890488



In [31]:
print("Testing  set score for model: %f" % classifier.score(test.drop(["YEAR","target","FL_DATE"], axis=1).values  , test[target] ))

Testing  set score for model: 0.491582
